**19장 – 대규모 텐서플로 모델 훈련과 배포**

_이 노트북에는 19장의 모든 샘플 코드와 연습 문제에 대한 해답이 포함되어 있습니다._

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/rickiepark/handson-ml3/blob/main/19_training_and_deploying_at_scale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
</table>

# 설정

이 프로젝트에는 Python 3.7 이상이 필요합니다:

In [1]:
import sys

assert sys.version_info >= (3, 7)

그리고 TensorFlow ≥ 2.8:

In [2]:
from packaging import version
import tensorflow as tf

assert version.parse(tf.__version__) >= version.parse("2.8.0")

코랩에서 실행하는 경우, 이 노트북의 뒷부분에서 사용하게 될 구글 AI 플랫폼 클라이언트 라이브러리를 설치해야 합니다. 버전 비호환성에 대한 경고는 무시해도 됩니다.

* **경고**: 코랩에서는 설치 후 런타임을 다시 시작하고 다음 셀을 계속 진행해야 합니다.

In [3]:
import sys
if "google.colab" in sys.modules or "kaggle_secrets" in sys.modules:
    %pip install -q -U google-cloud-aiplatform

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.0/321.0 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 77.4 MB/s eta 0:00:00


이 장에서는 하나 이상의 GPU에서 모델을 실행하거나 훈련하는 방법에 대해 설명하므로 적어도 하나 이상의 GPU가 있는지 확인하거나 그렇지 않으면 경고를 발행합니다:

In [4]:
if not tf.config.list_physical_devices('GPU'):
    print("GPU가 감지되지 않았습니다. 신경망은 GPU가 없으면 매우 느릴 수 있습니다.")
    if "google.colab" in sys.modules:
        print("런타임 > 런타임 유형 변경으로 이동하여 하드웨어 가속기로 GPU를 선택합니다.")

# 텐서플로 모델 서빙하기

먼저 TF 서빙을 사용하여 모델을 배포한 다음 Google Vertex AI에 배포해 보겠습니다.

## 텐서플로 서빙 사용

가장 먼저 해야 할 일은 모델을 빌드하고 학습한 다음 SavedModel 포맷으로 내보내는 것입니다.

### SavedModel 내보내기

MNIST 데이터 세트를 로드하고, 스케일을 조정하고, 분할해 보겠습니다.

In [5]:
from pathlib import Path
import tensorflow as tf

# 추가 코드 - MNIST 데이터 세트 로드 및 분할
mnist = tf.keras.datasets.mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = mnist
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

# 추가 코드 - MNIST 모델 구축 및 훈련(이미지 전처리도 처리)
tf.random.set_seed(42)
tf.keras.backend.clear_session()
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28], dtype=tf.uint8),
    tf.keras.layers.Rescaling(scale=1 / 255),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(learning_rate=1e-2),
              metrics=["accuracy"])
model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

model_name = "my_mnist_model"
model_version = "0001"
model_path = Path(model_name) / model_version
model.save(model_path, save_format="tf")

11490434/11490434 [==============================] - 1s 0us/step
Epoch 1/10
1719/1719 [==============================] - 17s 6ms/step - loss: 0.6884 - accuracy: 0.8215 - val_loss: 0.3751 - val_accuracy: 0.8962
Epoch 2/10
1719/1719 [==============================] - 10s 6ms/step - loss: 0.3543 - accuracy: 0.9013 - val_loss: 0.3003 - val_accuracy: 0.9148
Epoch 3/10
1719/1719 [==============================] - 9s 5ms/step - loss: 0.3005 - accuracy: 0.9158 - val_loss: 0.2630 - val_accuracy: 0.9256
Epoch 4/10
1719/1719 [==============================] - 9s 5ms/step - loss: 0.2677 - accuracy: 0.9249 - val_loss: 0.2394 - val_accuracy: 0.9334
Epoch 5/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.2439 - accuracy: 0.9322 - val_loss: 0.2205 - val_accuracy: 0.9398
Epoch 6/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.2255 - accuracy: 0.9365 - val_loss: 0.2069 - val_accuracy: 0.9420
Epoch 7/10
1719/1719 [==============================] - 5s 3ms/step -

파일 트리를 살펴보겠습니다(각 파일의 용도에 대해서는 10장에서 설명했습니다):

In [6]:
sorted([str(path) for path in model_path.parent.glob("**/*")])  # 추가 코드

['my_mnist_model/0001',
 'my_mnist_model/0001/assets',
 'my_mnist_model/0001/fingerprint.pb',
 'my_mnist_model/0001/keras_metadata.pb',
 'my_mnist_model/0001/saved_model.pb',
 'my_mnist_model/0001/variables',
 'my_mnist_model/0001/variables/variables.data-00000-of-00001',
 'my_mnist_model/0001/variables/variables.index']

SavedModel을 검사해 보겠습니다:

In [7]:
!saved_model_cli show --dir '{model_path}'

2023-08-16 08:44:21.480374: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
The given SavedModel contains the following tag-sets:
'serve'


In [8]:
!saved_model_cli show --dir '{model_path}' --tag_set serve

2023-08-16 08:44:26.556166: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
The given SavedModel MetaGraphDef contains SignatureDefs with the following keys:
SignatureDef key: "__saved_model_init_op"
SignatureDef key: "serving_default"


In [9]:
!saved_model_cli show --dir '{model_path}' --tag_set serve \
                      --signature_def serving_default

2023-08-16 08:44:29.997828: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
The given SavedModel SignatureDef contains the following input(s):
  inputs['flatten_input'] tensor_info:
      dtype: DT_UINT8
      shape: (-1, 28, 28)
      name: serving_default_flatten_input:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['dense_1'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 10)
      name: StatefulPartitionedCall:0
Method name is: tensorflow/serving/predict


더 자세한 내용을 보려면 다음 명령을 실행하세요:

```ipython
!saved_model_cli show --dir '{model_path}' --all
```

### 텐서플로 서빙 설치 및 시작하기

이 노트북을 코랩에서 실행하는 경우, 텐서플로 서버를 설치해야 합니다:

In [10]:
if "google.colab" in sys.modules:
    url = "https://storage.googleapis.com/tensorflow-serving-apt"
    src = "stable tensorflow-model-server tensorflow-model-server-universal"
    !echo 'deb {url} {src}' > /etc/apt/sources.list.d/tensorflow-serving.list
    !curl '{url}/tensorflow-serving.release.pub.gpg' | apt-key add -
    !apt update -q && apt-get install -y tensorflow-model-server
    %pip install -q -U tensorflow-serving-api==2.11.1

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0Warning: apt-key is deprecated. Manage keyring files in trusted.gpg.d instead (see apt-key(8)).
100  2943  100  2943    0     0   8603      0 --:--:-- --:--:-- --:--:--  8630
OK
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 https://storage.googleapis.com/tensorflow-serving-apt stable InRelease [3,026 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 https://storage.googleapis.com/t

`tensorflow_model_server`가 설치된 경우(예: Colab에서 이 노트북을 실행하는 경우) 다음 2개의 셀을 실행하여 서버를 시작하세요. 사용 중인 OS가 Windows인 경우, 터미널에서 `tensorflow_model_server` 명령을 실행하고 `${MODEL_DIR}`을 `my_mnist_model` 디렉터리의 전체 경로로 바꿔야 할 수 있습니다.

In [11]:
import os

os.environ["MODEL_DIR"] = str(model_path.parent.absolute())

In [12]:
%%bash --bg
tensorflow_model_server \
    --port=8500 \
    --rest_api_port=8501 \
    --model_name=my_mnist_model \
    --model_base_path="${MODEL_DIR}" >my_server.log 2>&1

개인 컴퓨터에서 이 노트북을 실행하는 경우, 도커를 사용해 TF 서빙을 설치하려면 먼저 [Docker](https://docs.docker.com/install/)가 설치되어 있는지 확인한 후 터미널에서 다음 명령을 실행하세요. `path/to/my_mnist_model`을 `my_mnist_model` 디렉토리의 적절한 절대 경로로 대체해야 하지만, 컨테이너 경로 `/models/my_mnist_model`은 수정하지 마세요.

```bash
docker pull tensorflow/serving  # 최신 TF 서빙 이미지 다운로드

docker run -it --rm -v "/path/to/my_mnist_model:/models/my_mnist_model" \
    -p 8500:8500 -p 8501:8501 -e MODEL_NAME=my_mnist_model tensorflow/serving
```

### REST API로 TF 서빙에 쿼리하기

다음으로 TF 서빙에 REST 쿼리를 전송해 보겠습니다:

In [13]:
import json

X_new = X_test[:3]  # 분류할 새로운 숫자 이미지가 3개 있다고 가정합니다.
request_json = json.dumps({
    "signature_name": "serving_default",
    "instances": X_new.tolist(),
})

In [14]:
request_json[:100] + "..." + request_json[-10:]

'{"signature_name": "serving_default", "instances": [[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..., 0, 0]]]}'

이제 텐서플로 서빙의 REST API를 사용하여 예측을 해보겠습니다:

In [16]:
import requests

server_url = "http://localhost:8501/v1/models/my_mnist_model:predict"
response = requests.post(server_url, data=request_json)
response.raise_for_status()  # 오류 발생 시 예외 발생
response = response.json()

In [17]:
import numpy as np

y_proba = np.array(response["predictions"])
y_proba.round(2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.99, 0.  , 0.  ],
       [0.  , 0.  , 0.99, 0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.98, 0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ]])

### gRPC API로 TF 서빙에 쿼리하기

In [ ]:
from tensorflow_serving.apis.predict_pb2 import PredictRequest

request = PredictRequest()
request.model_spec.name = model_name
request.model_spec.signature_name = "serving_default"
input_name = model.input_names[0]  # == "flatten_input"
request.inputs[input_name].CopyFrom(tf.make_tensor_proto(X_new))

In [ ]:
import grpc
from tensorflow_serving.apis import prediction_service_pb2_grpc

channel = grpc.insecure_channel('localhost:8500')
predict_service = prediction_service_pb2_grpc.PredictionServiceStub(channel)
response = predict_service.Predict(request, timeout=10.0)

응답을 텐서로 변환합니다:

In [ ]:
output_name = model.output_names[0]
outputs_proto = response.outputs[output_name]
y_proba = tf.make_ndarray(outputs_proto)

In [ ]:
y_proba.round(2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ],
       [0.  , 0.  , 0.99, 0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.97, 0.01, 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.  ]],
      dtype=float32)

클라이언트에 텐서플로 라이브러리가 포함되어 있지 않은 경우, 다음과 같이 응답을 넘파이 배열로 변환할 수 있습니다:

In [ ]:
# 추가 코드 - tf.make_ndarray() 사용을 피하는 방법을 보여줍니다.
output_name = model.output_names[0]
outputs_proto = response.outputs[output_name]
shape = [dim.size for dim in outputs_proto.tensor_shape.dim]
y_proba = np.array(outputs_proto.float_val).reshape(shape)
y_proba.round(2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ],
       [0.  , 0.  , 0.99, 0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.97, 0.01, 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.  ]])

### 새 모델 버전 배포

In [ ]:
# 추가 코드 - 새로운 MNIST 모델 버전 빌드 및 훈련
np.random.seed(42)
tf.random.set_seed(42)
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28], dtype=tf.uint8),
    tf.keras.layers.Rescaling(scale=1 / 255),
    tf.keras.layers.Dense(50, activation="relu"),
    tf.keras.layers.Dense(50, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(learning_rate=1e-2),
              metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 2s 931us/step - loss: 0.7039 - accuracy: 0.8056 - val_loss: 0.3418 - val_accuracy: 0.9042
Epoch 2/10
1719/1719 [==============================] - 1s 855us/step - loss: 0.3204 - accuracy: 0.9082 - val_loss: 0.2674 - val_accuracy: 0.9242
Epoch 3/10
1719/1719 [==============================] - 2s 883us/step - loss: 0.2650 - accuracy: 0.9235 - val_loss: 0.2227 - val_accuracy: 0.9368
Epoch 4/10
1719/1719 [==============================] - 1s 869us/step - loss: 0.2319 - accuracy: 0.9329 - val_loss: 0.2032 - val_accuracy: 0.9432
Epoch 5/10
1719/1719 [==============================] - 1s 870us/step - loss: 0.2089 - accuracy: 0.9399 - val_loss: 0.1833 - val_accuracy: 0.9482
Epoch 6/10
1719/1719 [==============================] - 1s 871us/step - loss: 0.1908 - accuracy: 0.9446 - val_loss: 0.1740 - val_accuracy: 0.9498
Epoch 7/10
1719/1719 [==============================] - 2s 873us/step - loss: 0.1756 - accuracy: 0.9490 - val_loss: 0.1605 -

In [ ]:
model_version = "0002"
model_path = Path(model_name) / model_version
model.save(model_path, save_format="tf")

INFO:tensorflow:Assets written to: my_mnist_model/0002/assets


파일 트리를 다시 한 번 살펴봅시다:

In [ ]:
sorted([str(path) for path in model_path.parent.glob("**/*")])  # 추가 코드

['my_mnist_model/0001',
 'my_mnist_model/0001/assets',
 'my_mnist_model/0001/keras_metadata.pb',
 'my_mnist_model/0001/saved_model.pb',
 'my_mnist_model/0001/variables',
 'my_mnist_model/0001/variables/variables.data-00000-of-00001',
 'my_mnist_model/0001/variables/variables.index',
 'my_mnist_model/0002',
 'my_mnist_model/0002/assets',
 'my_mnist_model/0002/keras_metadata.pb',
 'my_mnist_model/0002/saved_model.pb',
 'my_mnist_model/0002/variables',
 'my_mnist_model/0002/variables/variables.data-00000-of-00001',
 'my_mnist_model/0002/variables/variables.index']

**경고**: 텐서플로 서빙이 새 모델을 로드하기까지 잠시 기다려야 할 수 있습니다.

In [ ]:
import requests

server_url = "http://localhost:8501/v1/models/my_mnist_model:predict"

response = requests.post(server_url, data=request_json)
response.raise_for_status()
response = response.json()

In [ ]:
response.keys()

dict_keys(['predictions'])

In [ ]:
y_proba = np.array(response["predictions"])
y_proba.round(2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ],
       [0.  , 0.  , 0.99, 0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.99, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ]])

## 버텍스 AI에서 예측 서비스 만들기

책의 가이드를 따라 구글 클라우드 플랫폼 계정을 생성하고 버텍스 AI 및 클라우드 스토리지 API를 활성화하세요. 그런 다음, 코랩에서 이 노트북을 실행하는 경우 다음 셀을 실행하여 구글 클라우드 플랫폼에서 사용한 것과 동일한 구글 계정으로 인증하고 코랩이 데이터에 액세스할 수 있도록 권한을 부여할 수 있습니다.

**경고: 이 노트북을 신뢰하는 경우에만 이 작업을 수행하세요!**
* https://github.com/rickiepark/handson-ml3 에 있는 공식 노트북이 아닌 경우 특히 주의하세요: 코랩 URL은 https://colab.research.google.com/github/rickiepark/handson-ml3 으로 시작합니다. 그렇지 않으면 이 코드가 여러분의 데이터로 원하는 모든 작업을 수행할 수 있습니다.

코랩에서 이 노트북을 실행하지 않는 경우, 책의 가이드를 따라 서비스 계정을 만들고 해당 서비스 계정의 키를 생성한 다음, 이 노트북의 디렉터리에 다운로드하고 이름을 `my_service_account_key.json`으로 지정해야 합니다(또는 `GOOGLE_APPLICATION_CREDENTIALS` 환경 변수가 이 파일을 가리키도록 합니다).

In [ ]:
project_id = "my_project"  ##### 이를 프로젝트 ID로 변경합니다. #####

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()
else:
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "my_service_account_key.json"

In [ ]:
from google.cloud import storage

bucket_name = "my_bucket"  ##### 고유한 버킷 이름으로 변경합니다. #####
location = "us-central1"

storage_client = storage.Client(project=project_id)
bucket = storage_client.create_bucket(bucket_name, location=location)
#bucket = storage_client.bucket(bucket_name)  # 버킷을 재사용하는 경우

In [ ]:
def upload_directory(bucket, dirpath):
    dirpath = Path(dirpath)
    for filepath in dirpath.glob("**/*"):
        if filepath.is_file():
            blob = bucket.blob(filepath.relative_to(dirpath.parent).as_posix())
            blob.upload_from_filename(filepath)

upload_directory(bucket, "my_mnist_model")

In [ ]:
# 추가 코드 – upload_directory()의 훨씬 빠른 멀티 스레드 구현
#           타깃 경로의 prefix를 받고 출력 기능도 있습니다.

from concurrent import futures

def upload_file(bucket, filepath, blob_path):
    blob = bucket.blob(blob_path)
    blob.upload_from_filename(filepath)

def upload_directory(bucket, dirpath, prefix=None, max_workers=50):
    dirpath = Path(dirpath)
    prefix = prefix or dirpath.name
    with futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_filepath = {
            executor.submit(
                upload_file,
                bucket, filepath,
                f"{prefix}/{filepath.relative_to(dirpath).as_posix()}"
            ): filepath
            for filepath in sorted(dirpath.glob("**/*"))
            if filepath.is_file()
        }
        for future in futures.as_completed(future_to_filepath):
            filepath = future_to_filepath[future]
            try:
                result = future.result()
            except Exception as ex:
                print(f"{filepath!s:60} 업로드 에러: {ex}")  # f!s is str(f)
            else:
                print(f"{filepath!s:60} 업로드 완료", end="\r")

    print(f"{dirpath!s:60} 업로드 완료")

또는 구글 클라우드 CLI를 설치한 경우(코랩에는 이미 설치되어 있음) 다음 `gsutil` 명령을 사용할 수 있습니다:

In [ ]:
#!gsutil -m cp -r my_mnist_model gs://{bucket_name}/

In [ ]:
from google.cloud import aiplatform

server_image = "gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-8:latest"

aiplatform.init(project=project_id, location=location)
mnist_model = aiplatform.Model.upload(
    display_name="mnist",
    artifact_uri=f"gs://{bucket_name}/my_mnist_model/0001",
    serving_container_image_uri=server_image,
)

Creating Model
Create Model backing LRO: projects/522977795627/locations/us-central1/models/4798114811986575360/operations/53403898236370944
Model created. Resource name: projects/522977795627/locations/us-central1/models/4798114811986575360
To use this Model in another session:
model = aiplatform.Model('projects/522977795627/locations/us-central1/models/4798114811986575360')


**경고**: 이 셀은 버텍스 AI가 컴퓨팅 노드를 프로비저닝할 때까지 기다리므로 실행하는 데 몇 분 정도 걸릴 수 있습니다:

In [ ]:
endpoint = aiplatform.Endpoint.create(display_name="mnist-endpoint")

endpoint.deploy(
    mnist_model,
    min_replica_count=1,
    max_replica_count=5,
    machine_type="n1-standard-4",
    accelerator_type="NVIDIA_TESLA_K80",
    accelerator_count=1
)

Creating Endpoint
Create Endpoint backing LRO: projects/522977795627/locations/us-central1/endpoints/5133373499481522176/operations/4135354010494304256
Endpoint created. Resource name: projects/522977795627/locations/us-central1/endpoints/5133373499481522176
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/522977795627/locations/us-central1/endpoints/5133373499481522176')
Deploying Model projects/522977795627/locations/us-central1/models/4798114811986575360 to Endpoint : projects/522977795627/locations/us-central1/endpoints/5133373499481522176
Deploy Endpoint model backing LRO: projects/522977795627/locations/us-central1/endpoints/5133373499481522176/operations/388359120522051584
Endpoint model deployed. Resource name: projects/522977795627/locations/us-central1/endpoints/5133373499481522176


In [ ]:
response = endpoint.predict(instances=X_new.tolist())

In [ ]:
import numpy as np

np.round(response.predictions, 2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ],
       [0.  , 0.  , 0.99, 0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.97, 0.01, 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.  ]])

In [ ]:
endpoint.undeploy_all()  # 엔드포인트에서 모든 모델 배포 취소
endpoint.delete()

Undeploying Endpoint model: projects/522977795627/locations/us-central1/endpoints/5133373499481522176
Undeploy Endpoint model backing LRO: projects/522977795627/locations/us-central1/endpoints/5133373499481522176/operations/3579722406467469312
Endpoint model undeployed. Resource name: projects/522977795627/locations/us-central1/endpoints/5133373499481522176
Deleting Endpoint : projects/522977795627/locations/us-central1/endpoints/5133373499481522176
Delete Endpoint  backing LRO: projects/522977795627/locations/us-central1/operations/4738836360561950720
Endpoint deleted. . Resource name: projects/522977795627/locations/us-central1/endpoints/5133373499481522176


## 버텍스 AI에서 배치 예측 작업 실행하기

In [ ]:
batch_path = Path("my_mnist_batch")
batch_path.mkdir(exist_ok=True)
with open(batch_path / "my_mnist_batch.jsonl", "w") as jsonl_file:
    for image in X_test[:100].tolist():
        jsonl_file.write(json.dumps(image))
        jsonl_file.write("\n")

upload_directory(bucket, batch_path)

Uploaded my_mnist_batch                                              


In [ ]:
batch_prediction_job = mnist_model.batch_predict(
    job_display_name="my_batch_prediction_job",
    machine_type="n1-standard-4",
    starting_replica_count=1,
    max_replica_count=5,
    accelerator_type="NVIDIA_TESLA_K80",
    accelerator_count=1,
    gcs_source=[f"gs://{bucket_name}/{batch_path.name}/my_mnist_batch.jsonl"],
    gcs_destination_prefix=f"gs://{bucket_name}/my_mnist_predictions/",
    sync=True  # 완료될 때까지 기다리지 않으려면 False로 설정합니다.
)

Creating BatchPredictionJob
BatchPredictionJob created. Resource name: projects/522977795627/locations/us-central1/batchPredictionJobs/4346926367237996544
To use this BatchPredictionJob in another session:
bpj = aiplatform.BatchPredictionJob('projects/522977795627/locations/us-central1/batchPredictionJobs/4346926367237996544')
View Batch Prediction Job:
https://console.cloud.google.com/ai/platform/locations/us-central1/batch-predictions/4346926367237996544?project=522977795627
BatchPredictionJob projects/522977795627/locations/us-central1/batchPredictionJobs/4346926367237996544 current state:
JobState.JOB_STATE_PENDING
BatchPredictionJob projects/522977795627/locations/us-central1/batchPredictionJobs/4346926367237996544 current state:
JobState.JOB_STATE_RUNNING
BatchPredictionJob projects/522977795627/locations/us-central1/batchPredictionJobs/4346926367237996544 current state:
JobState.JOB_STATE_RUNNING
BatchPredictionJob projects/522977795627/locations/us-central1/batchPredictionJobs/

In [ ]:
batch_prediction_job.output_info  # 추가 코드 - 출력 디렉토리를 표시합니다.

gcs_output_directory: "gs://my_bucket/my_mnist_predictions/prediction-mnist-2022_04_12T21_30_08_071Z"

In [ ]:
y_probas = []
for blob in batch_prediction_job.iter_outputs():
    print(blob.name)  # 추가 코드
    if "prediction.results" in blob.name:
        for line in blob.download_as_text().splitlines():
            y_proba = json.loads(line)["prediction"]
            y_probas.append(y_proba)

my_mnist_predictions/prediction-mnist-2022_04_12T21_30_08_071Z/prediction.errors_stats-00000-of-00001
my_mnist_predictions/prediction-mnist-2022_04_12T21_30_08_071Z/prediction.results-00000-of-00002
my_mnist_predictions/prediction-mnist-2022_04_12T21_30_08_071Z/prediction.results-00001-of-00002


In [ ]:
y_pred = np.argmax(y_probas, axis=1)
accuracy = np.sum(y_pred == y_test[:100]) / 100

In [ ]:
accuracy

0.98

In [ ]:
mnist_model.delete()

Deleting Model : projects/522977795627/locations/us-central1/models/4798114811986575360
Delete Model  backing LRO: projects/522977795627/locations/us-central1/operations/598902403101622272
Model deleted. . Resource name: projects/522977795627/locations/us-central1/models/4798114811986575360


GCS에서 만든 모든 디렉토리(즉, 디렉토리 이름의 접두사를 가진 모든 블롭)를 삭제해 보겠습니다:

In [ ]:
for prefix in ["my_mnist_model/", "my_mnist_batch/", "my_mnist_predictions/"]:
    blobs = bucket.list_blobs(prefix=prefix)
    for blob in blobs:
        blob.delete()

#bucket.delete()  # 버킷 자체를 삭제하려면 주석을 제거하고 실행하세요.
batch_prediction_job.delete()

Deleting BatchPredictionJob : projects/522977795627/locations/us-central1/batchPredictionJobs/4346926367237996544
Delete BatchPredictionJob  backing LRO: projects/522977795627/locations/us-central1/operations/6699028098374959104
BatchPredictionJob deleted. . Resource name: projects/522977795627/locations/us-central1/batchPredictionJobs/4346926367237996544


# 모바일 또는 임베디드 디바이스에 모델 배포하기

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(str(model_path))
tflite_model = converter.convert()
with open("my_converted_savedmodel.tflite", "wb") as f:
    f.write(tflite_model)

2022-04-10 09:03:52.237094: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2022-04-10 09:03:52.237108: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2022-04-10 09:03:52.237830: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: my_mnist_model/0001
2022-04-10 09:03:52.238869: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2022-04-10 09:03:52.238881: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if present) from: my_mnist_model/0001
2022-04-10 09:03:52.242108: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2022-04-10 09:03:52.263868: I tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: my_mnist_model/0001
2022-04-10 09:03:52.271298: I tensorflow/cc/saved_model/loader.cc:301] SavedModel load for tags { serve }; Status: success: OK. Too

In [ ]:
# 추가 코드 - 케라스 모델을 변환하는 방법을 보여줍니다.
converter = tf.lite.TFLiteConverter.from_keras_model(model)

In [ ]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]

In [ ]:
tflite_model = converter.convert()
with open("my_converted_keras_model.tflite", "wb") as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /var/folders/wy/h39t6kb11pnbb0pzhksd_fqh0000gq/T/tmp6ffbc1qs/assets


INFO:tensorflow:Assets written to: /var/folders/wy/h39t6kb11pnbb0pzhksd_fqh0000gq/T/tmp6ffbc1qs/assets
2022-04-10 09:26:30.319286: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2022-04-10 09:26:30.319301: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2022-04-10 09:26:30.319417: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /var/folders/wy/h39t6kb11pnbb0pzhksd_fqh0000gq/T/tmp6ffbc1qs
2022-04-10 09:26:30.320420: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2022-04-10 09:26:30.320431: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if present) from: /var/folders/wy/h39t6kb11pnbb0pzhksd_fqh0000gq/T/tmp6ffbc1qs
2022-04-10 09:26:30.323773: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2022-04-10 09:26:30.345416: I tensorflow/cc/saved_model/loader.cc:212] Running initialization

# 웹 페이지에서 모델 실행하기

이 섹션의 코드 예제는 웹 앱을 무료로 만들 수 있는 웹사이트인 glitch.com에서 호스팅됩니다.

* https://homl.info/tfjscode: 사전 학습된 모델을 로드하고 이미지를 분류하는 간단한 TFJS 웹 앱입니다.
* https://homl.info/tfjswpa: WPA로 설정된 동일한 웹 앱. 모바일 장치를 포함한 다양한 플랫폼에서 이 링크를 열어 보세요.
** https://homl.info/wpacode: 이 WPA의 소스 코드입니다.
* https://tensorflow.org/js: TFJS 라이브러리.
** https://www.tensorflow.org/js/demos: 재미있는 데모 몇 가지를 소개합니다.

# GPU를 사용하여 계산 속도 향상하기

텐서플로우가 GPU를 볼 수 있는지 확인해 보겠습니다:

In [ ]:
physical_gpus = tf.config.list_physical_devices("GPU")
physical_gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


텐서플로 스크립트에서 GPU \#0과 \#1(PCI 순서 기준)만 사용하려면 스크립트를 시작하기 전에 환경 변수 `CUDA_DEVICE_ORDER=PCI_BUS_ID`와 `CUDA_VISIBLE_DEVICES=0,1`을 설정합니다. 또는 스크립트 자체에서 텐서플로를 사용하기 전에 지정할 수 있습니다.

## GPU RAM 관리

RAM 용량을 GPU당 2GB로 제한하려면:

In [ ]:
#for gpu in physical_gpus:
#    tf.config.set_logical_device_configuration(
#        gpu,
#        [tf.config.LogicalDeviceConfiguration(memory_limit=2048)]
#    )

점진적으로 텐서플로가 메모리를 점유하도록 하려면(프로세스가 종료될 때만 메모리를 해제합니다):

In [ ]:
#for gpu in physical_gpus:
#    tf.config.experimental.set_memory_growth(gpu, True)

이와 동일하게, 텐서플로를 사용하기 전에 `TF_FORCE_GPU_ALLOW_GROWTH` 환경 변수를 `true`로 설정할 수 있습니다.

물리적 GPU를 두 개의 논리적 GPU로 분할합니다:

In [ ]:
#tf.config.set_logical_device_configuration(
#    physical_gpus[0],
#    [tf.config.LogicalDeviceConfiguration(memory_limit=2048),
#     tf.config.LogicalDeviceConfiguration(memory_limit=2048)]
#)

In [ ]:
logical_gpus = tf.config.list_logical_devices("GPU")
logical_gpus

[LogicalDevice(name='/device:GPU:0', device_type='GPU')]


## 디바이스에 연산 및 변수 배치하기

모든 변수 및 연산 배치를 기록하려면(이 작업은 텐서플로를 임포팅한 직후에 실행해야 합니다):

In [ ]:
#tf.get_logger().setLevel("DEBUG")  # 디폴트 로그 수준은 INFO입니다.
#tf.debugging.set_log_device_placement(True)

In [ ]:
a = tf.Variable([1., 2., 3.])  # float32 변수는 GPU로 이동합니다.
a.device

'/job:localhost/replica:0/task:0/device:GPU:0'

In [ ]:
b = tf.Variable([1, 2, 3])  # int32 변수는 CPU로 이동합니다.
b.device

'/job:localhost/replica:0/task:0/device:CPU:0'

`tf.device()` 컨텍스트를 사용하여 원하는 장치에 변수 및 연산을 수동으로 배치할 수 있습니다:

In [ ]:
with tf.device("/cpu:0"):
    c = tf.Variable([1., 2., 3.])

c.device

'/job:localhost/replica:0/task:0/device:CPU:0'

존재하지 않거나 커널이 없는 장치를 지정하면 텐서플로는 자동으로 기본 장치를 사용합니다:

In [ ]:
# 추가 코드

with tf.device("/gpu:1234"):
    d = tf.Variable([1., 2., 3.])

d.device

"'/job:localhost/replica:0/task:0/device:GPU:0'"

텐서플로에서 존재하지 않는 장치를 사용하려고 할 때 기본 장치로 되돌아가지 않고 예외를 발생시키려면:

In [ ]:
tf.config.set_soft_device_placement(False)

# 추가 코드
try:
    with tf.device("/gpu:1000"):
        d = tf.Variable([1., 2., 3.])
except tf.errors.InvalidArgumentError as ex:
    print(ex)

tf.config.set_soft_device_placement(True)  # 추가 코드 - 소프트 배치로 돌아가기

Could not satisfy device specification '/job:localhost/replica:0/task:0/device:GPU:1000'. enable_soft_placement=0. Supported device types [CPU]. All available devices [/job:localhost/replica:0/task:0/device:CPU:0].


## 여러 디바이스에서 병렬 실행

inter-op 또는 intra-op 스레드 수를 설정하려는 경우(CPU 포화를 방지하거나 완벽하게 재현 가능한 테스트 케이스를 실행하기 위해 텐서플로를 단일 스레드로 만들고자 하는 경우 유용할 수 있습니다):

In [ ]:
#tf.config.threading.set_inter_op_parallelism_threads(10)
#tf.config.threading.set_intra_op_parallelism_threads(10)

# 여러 디바이스에서 모델 훈련하기

## 분산 전략 API를 사용한 대규모 훈련

In [ ]:
# 추가 코드 - 케라스를 사용하여 MNIST용 CNN 모델을 생성합니다.
def create_model():
    return tf.keras.Sequential([
        tf.keras.layers.Reshape([28, 28, 1], input_shape=[28, 28],
                                dtype=tf.uint8),
        tf.keras.layers.Rescaling(scale=1 / 255),
        tf.keras.layers.Conv2D(filters=64, kernel_size=7, activation="relu",
                               padding="same"),
        tf.keras.layers.MaxPooling2D(pool_size=2),
        tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu",
                               padding="same"),
        tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu",
                               padding="same"),
        tf.keras.layers.MaxPooling2D(pool_size=2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units=64, activation="relu"),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=10, activation="softmax"),
    ])

In [ ]:
tf.random.set_seed(42)

strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    model = create_model()  # 일반적인 케라스 모델을 생성합니다.
    model.compile(loss="sparse_categorical_crossentropy",
                  optimizer=tf.keras.optimizers.SGD(learning_rate=1e-2),
                  metrics=["accuracy"])  # 모델을 컴파일합니다.

batch_size = 100  # 복제본 수로 나누어지는 것이 바람직합니다.
model.fit(X_train, y_train, epochs=10,
          validation_data=(X_valid, y_valid), batch_size=batch_size)

In [ ]:
type(model.weights[0])

tensorflow.python.distribute.values.MirroredVariable

In [ ]:
model.predict(X_new).round(2)  # 추가 코드 - 배치가 모든 복제본에 분할됩니다.

array([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)


In [ ]:
# 추가 코드 - 모델을 저장해도 분산 전략이 보존되지 않음을 보여줍니다.
model.save("my_mirrored_model", save_format="tf")
model = tf.keras.models.load_model("my_mirrored_model")
type(model.weights[0])

INFO:tensorflow:Assets written to: my_mirrored_model/assets


tensorflow.python.ops.resource_variable_ops.ResourceVariable

In [ ]:
with strategy.scope():
    model = tf.keras.models.load_model("my_mirrored_model")

In [ ]:
type(model.weights[0])

tensorflow.python.distribute.values.MirroredVariable


사용할 GPU 리스트를 지정하려는 경우:

In [ ]:
strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


기본 all-reduce 알고리즘을 변경하려는 경우:

In [ ]:
strategy = tf.distribute.MirroredStrategy(
    cross_device_ops=tf.distribute.HierarchicalCopyAllReduce())

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


`CentralStorageStrategy`을 사용하려는 경우:

In [ ]:
strategy = tf.distribute.experimental.CentralStorageStrategy()

INFO:tensorflow:ParameterServerStrategy (CentralStorageStrategy if you are using a single machine) with compute_devices = ['/job:localhost/replica:0/task:0/device:CPU:0'], variable_device = '/job:localhost/replica:0/task:0/device:CPU:0'


In [ ]:
# Google Colab에서 TPU로 훈련하기:
#if "google.colab" in sys.modules and "COLAB_TPU_ADDR" in os.environ:
#  tpu_address = "grpc://" + os.environ["COLAB_TPU_ADDR"]
#else:
#  tpu_address = ""
#resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu_address)
#tf.config.experimental_connect_to_cluster(resolver)
#tf.tpu.experimental.initialize_tpu_system(resolver)
#strategy = tf.distribute.experimental.TPUStrategy(resolver)

## 텐서플로 클러스터에서 모델 훈련하기

텐서플로 클러스터는 일반적으로 서로 다른 컴퓨터에서 병렬로 실행되며 신경망 훈련이나 실행과 같은 작업을 완료하기 위해 서로 대화하는 텐서플로 프로세스의 그룹입니다. 클러스터의 각 TF 프로세스를 "태스크"(또는 "TF 서버")라고 합니다. IP 주소, 포트, 타입(역할 또는 작업이라고도 함)이 있습니다. 타입은 `"worker"`, `"chief"`, `"ps"`(파라미터 서버) 또는 `"evaluator"`가 될 수 있습니다:
* **워커**는 일반적으로 하나 이상의 GPU가 있는 컴퓨터에서 계산을 수행합니다.
* **치프**는 계산도 수행하지만, 텐서보드 로그 작성이나 체크포인트 저장과 같은 추가 작업도 처리합니다. 클러스터에는 하나의 치프가 있습니다. 정의되지 않은 경우 워커 #0이 치프가 됩니다.
* **파라미터 서버**(ps)는 변수 값만 추적하며, 일반적으로 CPU 전용 머신에 있습니다.
* **이밸류에이터**는 당연히 평가를 담당합니다. 일반적으로 클러스터에는 하나의 이밸류에이터가 있습니다.

같은 유형의 태스크 집합을 흔히 "작업(job)"이라고 합니다. 예를 들어 "워커" 작업은 모든 워커 태스크의 집합입니다.

텐서플로 클러스터를 시작하려면 먼저 클러스터를 정의해야 합니다. 즉, 모든 작업(IP 주소, TCP 포트 및 타입)을 지정해야 합니다. 예를 들어, 다음 클러스터 사양은 3개의 태스크(워커 2개, 파라미터 서버 1개)가 있는 클러스터를 정의합니다. 작업당 하나의 키가 있는 사전이며, 값은 태스크 주소 목록입니다:

In [ ]:
cluster_spec = {
    "worker": [
        "machine-a.example.com:2222",     # /job:worker/task:0
        "machine-b.example.com:2222"      # /job:worker/task:1
    ],
    "ps": ["machine-a.example.com:2221"]  # /job:ps/task:0
}

클러스터의 모든 태스크는 서버의 다른 모든 태스크와 통신할 수 있으므로 방화벽이 해당 포트에서 컴퓨터 간의 모든 통신을 승인하도록 구성해야 합니다(일반적으로 모든 컴퓨터에서 동일한 포트를 사용하는 것이 간단합니다).

태스크가 시작되면 어떤 태스크인지 타입과 인덱스(태스크 인덱스는 태스크 ID라고도 함)를 알려주어야 합니다. 클러스터 사양과 현재 태스크의 타입 및 아이디를 모두 한 번에 지정하는 일반적인 방법은 프로그램을 시작하기 전에 `TF_CONFIG` 환경 변수를 설정하는 것입니다. 클러스터 사양(``cluster`` 키 아래)과 시작할 태스크의 타입 및 인덱스(``task`` 키 아래)가 포함된 JSON 인코딩된 딕셔너리여야 합니다. 예를 들어, 다음 `TF_CONFIG` 환경 변수는 위와 동일한 클러스터(워커 2개, 파라미터 서버 1개)를 정의하고, 시작할 태스크를 워커 \#0으로 지정합니다:

In [ ]:
os.environ["TF_CONFIG"] = json.dumps({
    "cluster": cluster_spec,
    "task": {"type": "worker", "index": 0}
})

일부 플랫폼(예: 구글 버텍스 AI)에서는 이 환경 변수를 자동으로 설정합니다.

텐서플로의 `TFConfigClusterResolver` 클래스는 이 환경 변수에서 클러스터 구성을 읽습니다:

In [ ]:
resolver = tf.distribute.cluster_resolver.TFConfigClusterResolver()
resolver.cluster_spec()

ClusterSpec({'ps': ['machine-a.example.com:2221'], 'worker': ['machine-a.example.com:2222', 'machine-b.example.com:2222']})

In [ ]:
resolver.task_type

'worker'

In [ ]:
resolver.task_id

0

이제 로컬 컴퓨터에서 두 개의 워커 태스크를 가진 간단한 클러스터를 실행해 보겠습니다. `MultiWorkerMirroredStrategy`을 사용하여 두 태스크로 모델을 훈련하겠습니다.

첫 번째 단계는 훈련 코드를 작성하는 것입니다. 이 코드를 사용해 두 워커에서 각각 고유한 프로세스로 실행하므로 별도의 파이썬 파일인 `my_mnist_multiworker_task.py`에 작성합니다. 코드는 비교적 간단하지만 주의해야 할 몇 가지 중요한 사항이 있습니다:
* 텐서플로로 다른 작업을 수행하기 전에 `MultiWorkerMirroredStrategy`을 생성합니다.
* 워커 중 하나만 텐서보드 로깅을 처리합니다. 앞서 언급했듯이 이 작업자를 *치프*라고 합니다. 명시적으로 정의되지 않은 경우 워커 #0이 치프가 됩니다.

In [ ]:
%%writefile my_mnist_multiworker_task.py

import tempfile
import tensorflow as tf

strategy = tf.distribute.MultiWorkerMirroredStrategy()  # 시작 부분에!
resolver = tf.distribute.cluster_resolver.TFConfigClusterResolver()
print(f"Starting task {resolver.task_type} #{resolver.task_id}")

# 추가 코드 - MNIST 데이터셋 로드 및 분할
mnist = tf.keras.datasets.mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = mnist
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

with strategy.scope():
    model = tf.keras.Sequential([
        tf.keras.layers.Reshape([28, 28, 1], input_shape=[28, 28],
                                dtype=tf.uint8),
        tf.keras.layers.Rescaling(scale=1 / 255),
        tf.keras.layers.Conv2D(filters=64, kernel_size=7, activation="relu",
                               padding="same", input_shape=[28, 28, 1]),
        tf.keras.layers.MaxPooling2D(pool_size=2),
        tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu",
                               padding="same"),
        tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu",
                               padding="same"),
        tf.keras.layers.MaxPooling2D(pool_size=2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units=64, activation="relu"),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=10, activation="softmax"),
    ])
    model.compile(loss="sparse_categorical_crossentropy",
                  optimizer=tf.keras.optimizers.SGD(learning_rate=1e-2),
                  metrics=["accuracy"])

model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=10)

if resolver.task_id == 0:  # 치프는 모델을 올바른 위치에 저장합니다.
    model.save("my_mnist_multiworker_model", save_format="tf")
else:
    tmpdir = tempfile.mkdtemp()  # 다른 워커는 임시 디렉터리에 저장합니다.
    model.save(tmpdir, save_format="tf")
    tf.io.gfile.rmtree(tmpdir)  # 마지막에 이 디렉터리를 삭제할 수 있습니다!

Writing my_mnist_multiworker_task.py


실제 애플리케이션에서는 일반적으로 머신당 하나의 워커가 있지만, 이 예제에서는 동일한 머신에서 두 워커를 모두 실행하고 있으므로 두 워커 모두 사용 가능한 모든 GPU RAM(이 머신에 GPU가 있는 경우)을 사용하려고 시도하므로 메모리 부족(OOM) 오류가 발생할 수 있습니다. 이를 방지하기 위해 `CUDA_VISIBLE_DEVICES` 환경 변수를 사용하여 각 워커에 다른 GPU를 할당할 수 있습니다. 또는 `CUDA_VISIBLE_DEVICES`를 빈 문자열로 설정하여 간단히 GPU 지원을 비활성화할 수 있습니다.

이제 각각 고유한 프로세스에서 두 워커를 시작할 준비가 되었습니다. 태스크 인덱스가 변경된 것을 확인할 수 있습니다:

In [ ]:
%%bash --bg

export CUDA_VISIBLE_DEVICES=''
export TF_CONFIG='{"cluster": {"worker": ["127.0.0.1:9901", "127.0.0.1:9902"]},
                   "task": {"type": "worker", "index": 0}}'
python my_mnist_multiworker_task.py > my_worker_0.log 2>&1

In [ ]:
%%bash --bg

export CUDA_VISIBLE_DEVICES=''
export TF_CONFIG='{"cluster": {"worker": ["127.0.0.1:9901", "127.0.0.1:9902"]},
                   "task": {"type": "worker", "index": 1}}'
python my_mnist_multiworker_task.py > my_worker_1.log 2>&1

**참고**: `AutoShardPolicy`에 대한 경고가 표시되면 무시해도 무방합니다. 자세한 내용은 [TF 이슈 #42146](https://github.com/tensorflow/tensorflow/issues/42146)을 참고하세요.

끝났습니다! 이제 텐서플로 클러스터가 실행 중이지만 별도의 프로세스에서 실행 중이므로 이 노트북에서는 볼 수 없습니다(하지만 진행 상황은 `my_worker_*.log`에서 확인할 수 있습니다).

치프(워커 #0)가 텐서보드에 로그를 쓰기 때문에, 훈련 진행 상황을 보기 위해 텐서보드를 사용할 수 있습니다. 다음 셀을 실행한 다음, 설정 버튼(즉, 톱니바퀴 아이콘)을 클릭하고 "Reload data" 상자를 체크하여 텐서보드가 30초마다 자동으로 새로고침되도록 설정합니다. 첫 번째 훈련이 완료되고(몇 분 정도 소요될 수 있음) 텐서보드가 새로 고침되면 SCALARS 탭이 나타납니다. 이 탭을 클릭하면 모델의 훈련 진행 과정과 검증 정확도를 확인할 수 있습니다.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=./my_mnist_multiworker_logs --port=6006

In [ ]:
# strategy = tf.distribute.MultiWorkerMirroredStrategy(
#     communication_options=tf.distribute.experimental.CommunicationOptions(
#         implementation=tf.distribute.experimental.CollectiveCommunication.NCCL))

## 버텍스 AI에서 대규모 훈련 작업 실행하기

훈련 스크립트를 복사하되 `import os`를 추가하고 저장 경로를 `AIP_MODEL_DIR` 환경 변수가 가리키는 GCS 경로로 변경해 보겠습니다:

In [ ]:
%%writefile my_vertex_ai_training_task.py

import os
from pathlib import Path
import tempfile
import tensorflow as tf

strategy = tf.distribute.MultiWorkerMirroredStrategy()  # 시작 부분에!
resolver = tf.distribute.cluster_resolver.TFConfigClusterResolver()

if resolver.task_type == "chief":
    model_dir = os.getenv("AIP_MODEL_DIR")  # 버텍스 AI가 제공하는 경로
    tensorboard_log_dir = os.getenv("AIP_TENSORBOARD_LOG_DIR")
    checkpoint_dir = os.getenv("AIP_CHECKPOINT_DIR")
else:
    tmp_dir = Path(tempfile.mkdtemp())  # 다른 워커는 임시 디렉터리를 사용합니다.
    model_dir = tmp_dir / "model"
    tensorboard_log_dir = tmp_dir / "logs"
    checkpoint_dir = tmp_dir / "ckpt"

callbacks = [tf.keras.callbacks.TensorBoard(tensorboard_log_dir),
             tf.keras.callbacks.ModelCheckpoint(checkpoint_dir)]

# 추가 코드 - MNIST 데이터셋 로드 및 준비
mnist = tf.keras.datasets.mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = mnist
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

# 추가 코드 - 분산 전략을 사용하여 케라스 모델을 빌드하고 컴파일합니다.
with strategy.scope():
    model = tf.keras.Sequential([
        tf.keras.layers.Reshape([28, 28, 1], input_shape=[28, 28],
                                dtype=tf.uint8),
        tf.keras.layers.Lambda(lambda X: X / 255),
        tf.keras.layers.Conv2D(filters=64, kernel_size=7, activation="relu",
                               padding="same", input_shape=[28, 28, 1]),
        tf.keras.layers.MaxPooling2D(pool_size=2),
        tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu",
                               padding="same"),
        tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu",
                               padding="same"),
        tf.keras.layers.MaxPooling2D(pool_size=2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units=64, activation="relu"),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=10, activation="softmax"),
    ])
    model.compile(loss="sparse_categorical_crossentropy",
                  optimizer=tf.keras.optimizers.SGD(learning_rate=1e-2),
                  metrics=["accuracy"])

model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=10,
          callbacks=callbacks)
model.save(model_dir, save_format="tf")

Writing my_vertex_ai_training_task.py


In [ ]:
custom_training_job = aiplatform.CustomTrainingJob(
    display_name="my_custom_training_job",
    script_path="my_vertex_ai_training_task.py",
    container_uri="gcr.io/cloud-aiplatform/training/tf-gpu.2-4:latest",
    model_serving_container_image_uri=server_image,
    requirements=["gcsfs==2022.3.0"],  # 필요 없음, 이것은 단지 예일 뿐입니다.
    staging_bucket=f"gs://{bucket_name}/staging"
)

In [ ]:
mnist_model2 = custom_training_job.run(
    machine_type="n1-standard-4",
    replica_count=2,
    accelerator_type="NVIDIA_TESLA_K80",
    accelerator_count=2,
)

Training script copied to:
gs://my_bucket/aiplatform-2022-04-14-10:08:24.124-aiplatform_custom_trainer_script-0.1.tar.gz.
Training Output directory:
gs://my_bucket/aiplatform-custom-training-2022-04-14-10:08:25.226 
View Training:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/5407999068506947584?project=522977795627
CustomTrainingJob projects/522977795627/locations/us-central1/trainingPipelines/5407999068506947584 current state:
PipelineState.PIPELINE_STATE_PENDING
CustomTrainingJob projects/522977795627/locations/us-central1/trainingPipelines/5407999068506947584 current state:
PipelineState.PIPELINE_STATE_RUNNING
View backing custom job:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/6685701948726837248?project=522977795627
CustomTrainingJob projects/522977795627/locations/us-central1/trainingPipelines/5407999068506947584 current state:
PipelineState.PIPELINE_STATE_RUNNING
CustomTrainingJob projects/522977795627/locations/us-c

정리합니다:

In [ ]:
mnist_model2.delete()
custom_training_job.delete()
blobs = bucket.list_blobs(prefix=f"gs://{bucket_name}/staging/")
for blob in blobs:
    blob.delete()

# 버텍스 AI의 하이퍼파라미터 튜닝

In [ ]:
%%writefile my_vertex_ai_trial.py

import argparse

parser = argparse.ArgumentParser()
parser.add_argument("--n_hidden", type=int, default=2)
parser.add_argument("--n_neurons", type=int, default=256)
parser.add_argument("--learning_rate", type=float, default=1e-2)
parser.add_argument("--optimizer", default="adam")
args = parser.parse_args()

import tensorflow as tf

def build_model(args):
    with tf.distribute.MirroredStrategy().scope():
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.Flatten(input_shape=[28, 28], dtype=tf.uint8))
        for _ in range(args.n_hidden):
            model.add(tf.keras.layers.Dense(args.n_neurons, activation="relu"))
        model.add(tf.keras.layers.Dense(10, activation="softmax"))
        opt = tf.keras.optimizers.get(args.optimizer)
        opt.learning_rate = args.learning_rate
        model.compile(loss="sparse_categorical_crossentropy", optimizer=opt,
                      metrics=["accuracy"])
        return model

# 추가 코드 - 데이터셋 로드 및 분할
mnist = tf.keras.datasets.mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = mnist
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

# 추가 코드 - AIP_* 환경 변수를 사용하고 콜백을 만듭니다.
import os
model_dir = os.getenv("AIP_MODEL_DIR")
tensorboard_log_dir = os.getenv("AIP_TENSORBOARD_LOG_DIR")
checkpoint_dir = os.getenv("AIP_CHECKPOINT_DIR")
trial_id = os.getenv("CLOUD_ML_TRIAL_ID")
tensorboard_cb = tf.keras.callbacks.TensorBoard(tensorboard_log_dir)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=5)
callbacks = [tensorboard_cb, early_stopping_cb]

model = build_model(args)
history = model.fit(X_train, y_train, validation_data=(X_valid, y_valid),
                    epochs=10, callbacks=callbacks)
model.save(model_dir, save_format="tf")  # 추가 코드

import hypertune

hypertune = hypertune.HyperTune()
hypertune.report_hyperparameter_tuning_metric(
    hyperparameter_metric_tag="accuracy",  # 보고할 지표의 이름
    metric_value=max(history.history["val_accuracy"]),  # 최대 정확도 값
    global_step=model.optimizer.iterations.numpy(),
)

Writing my_vertex_ai_trial.py


In [ ]:
trial_job = aiplatform.CustomJob.from_local_script(
    display_name="my_search_trial_job",
    script_path="my_vertex_ai_trial.py",  # 훈련 스크립트 경로
    container_uri="gcr.io/cloud-aiplatform/training/tf-gpu.2-4:latest",
    staging_bucket=f"gs://{bucket_name}/staging",
    accelerator_type="NVIDIA_TESLA_K80",
    accelerator_count=2,  # 이 예제에서는 각 트라이얼에 2개의 GPU가 있습니다.
)

Training script copied to:
gs://homl3-mybucket5/staging/aiplatform-2022-04-18-18:14:02.860-aiplatform_custom_trainer_script-0.1.tar.gz.


In [ ]:
from google.cloud.aiplatform import hyperparameter_tuning as hpt

hp_job = aiplatform.HyperparameterTuningJob(
    display_name="my_hp_search_job",
    custom_job=trial_job,
    metric_spec={"accuracy": "maximize"},
    parameter_spec={
        "learning_rate": hpt.DoubleParameterSpec(min=1e-3, max=10, scale="log"),
        "n_neurons": hpt.IntegerParameterSpec(min=1, max=300, scale="linear"),
        "n_hidden": hpt.IntegerParameterSpec(min=1, max=10, scale="linear"),
        "optimizer": hpt.CategoricalParameterSpec(["sgd", "adam"]),
    },
    max_trial_count=100,
    parallel_trial_count=20,
)
hp_job.run()

Creating HyperparameterTuningJob
HyperparameterTuningJob created. Resource name: projects/522977795627/locations/us-central1/hyperparameterTuningJobs/5825136187899117568
To use this HyperparameterTuningJob in another session:
hpt_job = aiplatform.HyperparameterTuningJob.get('projects/522977795627/locations/us-central1/hyperparameterTuningJobs/5825136187899117568')
View HyperparameterTuningJob:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/5825136187899117568?project=522977795627
HyperparameterTuningJob projects/522977795627/locations/us-central1/hyperparameterTuningJobs/5825136187899117568 current state:
JobState.JOB_STATE_RUNNING
HyperparameterTuningJob projects/522977795627/locations/us-central1/hyperparameterTuningJobs/5825136187899117568 current state:
JobState.JOB_STATE_RUNNING
HyperparameterTuningJob projects/522977795627/locations/us-central1/hyperparameterTuningJobs/5825136187899117568 current state:
JobState.JOB_STATE_RUNNING
HyperparameterTuningJ

In [ ]:
def get_final_metric(trial, metric_id):
    for metric in trial.final_measurement.metrics:
        if metric.metric_id == metric_id:
            return metric.value

trials = hp_job.trials
trial_accuracies = [get_final_metric(trial, "accuracy") for trial in trials]
best_trial = trials[np.argmax(trial_accuracies)]

In [ ]:
max(trial_accuracies)

0.977400004863739

In [ ]:
best_trial.id

'98'

In [ ]:
best_trial.parameters

[parameter_id: "learning_rate"
value {
  number_value: 0.001
}
, parameter_id: "n_hidden"
value {
  number_value: 8.0
}
, parameter_id: "n_neurons"
value {
  number_value: 216.0
}
, parameter_id: "optimizer"
value {
  string_value: "adam"
}
]

# 추가 자료 - 버텍스 AI의 분산 케라스 튜너

버텍스 AI의 하이퍼파라미터 튜닝 서비스를 사용하는 대신, 10장에서 소개한 [케라스 튜너](https://keras.io/keras_tuner/)를 사용하여 버텍스 AI VM에서 실행할 수 있습니다. 케라스 튜너는 하이퍼파라미터 탐색을 여러 머신에 분산하여 간단하게 확장할 수 있는 방법을 제공합니다: 각 머신에서 세 개의 환경 변수를 설정한 다음 일반 케라스 튜너 코드를 실행하기만 하면 됩니다. 모든 머신에서 똑같은 스크립트를 사용할 수 있습니다. 머신 중 한 대는 치프 역할을 하고 다른 머신은 워커 역할을 합니다. 각 워커는 오라클 역할을 하는 치프에게 어떤 하이퍼파라미터 값을 시도할지 요청하고, 치프는 이 하이퍼파라미터 값을 사용하여 모델을 훈련시킨 다음, 최종적으로 모델의 성능을 치프에게 보고하면 치프는 다음에 워커가 시도할 하이퍼파라미터 값을 결정할 수 있습니다.

각 머신에서 설정해야 하는 세 가지 환경 변수는 다음과 같습니다:

* `KERASTUNER_TUNER_ID`: 치프 머신은 `"chief"`이고 워커 머신은 `"worker0"`, `"worker1"` 등과 같은 고유 식별자입니다.
* `KERASTUNER_ORACLE_IP`: 치프 머신의 IP 주소 또는 호스트 이름입니다. 치프 자체는 일반적으로 `"0.0.0.0"`을 사용하여 머신의 모든 IP 주소에서 수신해야 합니다.
* `KERASTUNER_ORACLE_PORT`: 치프가 수신 대기할 TCP 포트입니다.

분산 케라스 튜너는 모든 머신 집합에서 사용할 수 있습니다. 버텍스 AI 머신에서 실행하려면 일반 훈련 작업을 생성하고 훈련 스크립트를 수정하여 세 가지 환경 변수를 적절히 설정한 후 케라스 튜너를 사용하면 됩니다.

예를 들어, 아래 스크립트는 이전과 마찬가지로 먼저 버텍스 AI가 자동으로 설정하는 `TF_CONFIG` 환경 변수를 파싱합니다. `"chief"` 타입의 태스크 주소를 찾아서 IP 주소 또는 호스트 이름과 TCP 포트를 추출합니다. 그런 다음 튜너 ID를 태스크 타입과 태스크 인덱스로 정의합니다(예: `"worker0"`). 튜너 ID가 `"chief0"`이면 `"chief"`로 변경하고 IP를 `"0.0.0.0"`으로 설정하면 해당 컴퓨터의 모든 IPv4 주소에서 수신 대기하게 됩니다. 그런 다음 케라스 튜너에 대한 환경 변수를 정의합니다. 다음으로 스크립트는 10장에서와 마찬가지로 튜너를 생성하여 탐색을 수행한 다음 마지막으로 버텍스 AI가 지정한 위치에 최적의 모델을 저장합니다:

In [ ]:
%%writefile my_keras_tuner_search.py

import json
import os

tf_config = json.loads(os.environ["TF_CONFIG"])

chief_ip, chief_port = tf_config["cluster"]["chief"][0].rsplit(":", 1)
tuner_id = f'{tf_config["task"]["type"]}{tf_config["task"]["index"]}'
if tuner_id == "chief0":
    tuner_id = "chief"
    chief_ip = "0.0.0.0"
    # 추가 코드 - 치프는 많은 작업을 수행하지 않으므로 동일한 컴퓨터에서 워커를 실행하여
    # 컴퓨팅 리소스를 최적화할 수 있습니다. 이렇게 하려면
    # TF_CONFIG 환경 변수를 조정하여 태스크 유형을 "worker"로 설정하고 태스크 인덱스를
    # 고유한 값으로 설정한 후 치프가 또 다른 프로세스를 시작하도록 하면 됩니다.
    # 다음 몇 줄의 주석 처리를 해제하고 시도해 보세요:
    # import subprocess
    # import sys
    # tf_config["task"]["type"] = "workerX"  # 치프 머신의 워커
    # os.environ["TF_CONFIG"] = json.dumps(tf_config)
    # subprocess.Popen([sys.executable] + sys.argv,
    #                  stdout=sys.stdout, stderr=sys.stderr)

os.environ["KERASTUNER_TUNER_ID"] = tuner_id
os.environ["KERASTUNER_ORACLE_IP"] = chief_ip
os.environ["KERASTUNER_ORACLE_PORT"] = chief_port

from pathlib import Path
import keras_tuner as kt
import tensorflow as tf

gcs_path = "/gcs/my_bucket/my_hp_search"  # 버킷 이름으로 바꾸기

def build_model(hp):
    n_hidden = hp.Int("n_hidden", min_value=0, max_value=8, default=2)
    n_neurons = hp.Int("n_neurons", min_value=16, max_value=256)
    learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-2,
                             sampling="log")
    optimizer = hp.Choice("optimizer", values=["sgd", "adam"])
    if optimizer == "sgd":
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    else:
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten(input_shape=[28, 28], dtype=tf.uint8))
    for _ in range(n_hidden):
        model.add(tf.keras.layers.Dense(n_neurons, activation="relu"))
    model.add(tf.keras.layers.Dense(10, activation="softmax"))
    model.compile(loss="sparse_categorical_crossentropy",
                  optimizer=optimizer,
                  metrics=["accuracy"])
    return model

hyperband_tuner = kt.Hyperband(
    build_model, objective="val_accuracy", seed=42,
    max_epochs=10, factor=3, hyperband_iterations=2,
    distribution_strategy=tf.distribute.MirroredStrategy(),
    directory=gcs_path, project_name="mnist")

# extra code – Load and split the MNIST dataset
mnist = tf.keras.datasets.mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = mnist
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

tensorboard_log_dir = os.environ["AIP_TENSORBOARD_LOG_DIR"] + "/" + tuner_id
tensorboard_cb = tf.keras.callbacks.TensorBoard(tensorboard_log_dir)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=5)
hyperband_tuner.search(X_train, y_train, epochs=10,
                       validation_data=(X_valid, y_valid),
                       callbacks=[tensorboard_cb, early_stopping_cb])

if tuner_id == "chief":
    best_hp = hyperband_tuner.get_best_hyperparameters()[0]
    best_model = hyperband_tuner.hypermodel.build(best_hp)
    best_model.save(os.getenv("AIP_MODEL_DIR"), save_format="tf")

Writing my_keras_tuner_search.py


참고로 버텍스 AI는 오픈 소스 [GCS 퓨즈 어댑터](https://cloud.google.com/storage/docs/gcs-fuse)를 사용하여 `/gcs` 디렉터리를 GCS에 자동으로 마운트합니다. 이렇게 하면 워커와 치프 간에 공유 디렉터리가 제공되며, 이는 케라스 튜너에 필요합니다. 또한 배포 전략을 `MirroredStrategy`으로 설정했습니다. 이렇게 하면 각 워커가 자신의 머신에 있는 모든 GPU를 사용할 수 있습니다(GPU가 두 개 이상 있는 경우).

`gcs/my_bucket/`를 <code>/gcs/<i>{bucket_name}</i>/</code>로 바꿉니다:

In [ ]:
with open("my_keras_tuner_search.py") as f:
    script = f.read()

with open("my_keras_tuner_search.py", "w") as f:
    f.write(script.replace("/gcs/my_bucket/", f"/gcs/{bucket_name}/"))

이제 이전 섹션에서와 똑같이 이 스크립트를 기반으로 사용자 정의 훈련 작업을 시작하기만 하면 됩니다. `requirements` 목록에 `keras-tuner`를 추가하는 것을 잊지 마세요:

In [ ]:
hp_search_job = aiplatform.CustomTrainingJob(
    display_name="my_hp_search_job",
    script_path="my_keras_tuner_search.py",
    container_uri="gcr.io/cloud-aiplatform/training/tf-gpu.2-4:latest",
    model_serving_container_image_uri=server_image,
    requirements=["keras-tuner~=1.1.2"],
    staging_bucket=f"gs://{bucket_name}/staging",
)

In [ ]:
mnist_model3 = hp_search_job.run(
    machine_type="n1-standard-4",
    replica_count=3,
    accelerator_type="NVIDIA_TESLA_K80",
    accelerator_count=2,
)

Training script copied to:
gs://my_bucket/staging/aiplatform-2022-04-15-13:34:32.591-aiplatform_custom_trainer_script-0.1.tar.gz.
Training Output directory:
gs://my_bucket/staging/aiplatform-custom-training-2022-04-15-13:34:34.453 
View Training:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/8601543785521872896?project=522977795627
View backing custom job:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/5022607048831926272?project=522977795627
CustomTrainingJob projects/522977795627/locations/us-central1/trainingPipelines/8601543785521872896 current state:
PipelineState.PIPELINE_STATE_RUNNING
CustomTrainingJob projects/522977795627/locations/us-central1/trainingPipelines/8601543785521872896 current state:
PipelineState.PIPELINE_STATE_RUNNING
CustomTrainingJob projects/522977795627/locations/us-central1/trainingPipelines/8601543785521872896 current state:
PipelineState.PIPELINE_STATE_RUNNING
CustomTrainingJob projects/52297779562

모델을 찾았습니다!

정리:

In [ ]:
mnist_model3.delete()
hp_search_job.delete()
blobs = bucket.list_blobs(prefix=f"gs://{bucket_name}/staging/")
for blob in blobs:
    blob.delete()

# 추가 자료 - AutoML을 사용하여 모델 훈련하기

먼저 MNIST 데이터셋을 PNG 이미지로 내보내고, 각 이미지 파일과 분할(훈련, 검증 또는 테스트), 레이블을 나타내는 `import.csv`를 준비해 보겠습니다:

In [ ]:
import matplotlib.pyplot as plt

mnist_path = Path("datasets/mnist")
mnist_path.mkdir(parents=True, exist_ok=True)
idx = 0
with open(mnist_path / "import.csv", "w") as import_csv:
    for split, X, y in zip(("training", "validation", "test"),
                           (X_train, X_valid, X_test),
                           (y_train, y_valid, y_test)):
        for image, label in zip(X, y):
            print(f"\r{idx + 1}/70000", end="")
            filename = f"{idx:05d}.png"
            plt.imsave(mnist_path / filename, np.tile(image, 3))
            line = f"{split},gs://{bucket_name}/mnist/{filename},{label}\n"
            import_csv.write(line)
            idx += 1

70000/70000

이 데이터셋을 GCS에 업로드해 보겠습니다:

In [ ]:
upload_directory(bucket, mnist_path)

Uploaded datasets/mnist                                              


이제 버텍스 AI에서 관리용 이미지 데이터셋을 만들어 보겠습니다:

In [ ]:
from aiplatform.schema.dataset.ioformat.image import single_label_classification

mnist_dataset = aiplatform.ImageDataset.create(
    display_name="mnist-dataset",
    gcs_source=[f"gs://{bucket_name}/mnist/import.csv"],
    project=project_id,
    import_schema_uri=single_label_classification,
    sync=True,
)

Creating ImageDataset
Create ImageDataset backing LRO: projects/522977795627/locations/us-central1/datasets/7532459492777132032/operations/3812233931370004480
ImageDataset created. Resource name: projects/522977795627/locations/us-central1/datasets/7532459492777132032
To use this ImageDataset in another session:
ds = aiplatform.ImageDataset('projects/522977795627/locations/us-central1/datasets/7532459492777132032')
Importing ImageDataset data: projects/522977795627/locations/us-central1/datasets/7532459492777132032
Import ImageDataset data backing LRO: projects/522977795627/locations/us-central1/datasets/7532459492777132032/operations/3010593197698056192
ImageDataset data imported. Resource name: projects/522977795627/locations/us-central1/datasets/7532459492777132032


이 데이터셋에 대해 AutoML 학습 작업을 생성합니다:

**TODO**

# 연습문제 해답

## 1. to 8.

부록 A 참조

## 9.
_문제: (원하는 어떤 모델이든) 모델을 훈련하고 TF 서빙이나 구글 버텍스 AI 플랫폼에 배포해보세요. REST API나 gRPC API를 사용해 쿼리하는 클라이언트 코드를 작성해보세요. 모델을 업데이트하고 새로운 버전을 배포해보세요. 클라이언트 코드가 새로운 버전으로 쿼리할 것입니다. 첫 번째 버전으로 롤백해보세요._

<a href="#텐서플로-모델-서빙하기">텐서플로 모델 서빙하기</a> 절에 있는 단계를 따라해 보세요.

# 10.
_문제: 하나의 머신에 여러 개의 GPU에서 `MirroredStrategy` 전략으로 모델을 훈련해보세요(GPU를 준비하지 못하면 코랩의 GPU 런타임을 사용하여 가상 GPU 2개를 만들 수 있습니다). `CentralStorageStrategy` 전략으로 모델을 다시 훈련하고 훈련 시간을 비교해보세요._

[여러 디바이스에서 모델 훈련하기](#여러-디바이스에서-모델-훈련하기) 절에 있는 단계를 따라해 보세요.

# 11.
_문제: 케라스 튜너 또는 버텍스 AI의 하이퍼파라미터 튜닝 서비스를 사용하여 버텍스 AI에서 원하는 모델을 세부 튜닝해 보세요._

이 책의 _케라스 튜너를 사용한 하이퍼파라미터 튜닝_ 섹션에 있는 지침을 따르세요.

# 축하합니다!

책의 마지막에 도달했습니다! 도움이 되셨기를 바랍니다. 😊